# Preprocessing 

In [3]:
# import docx
import pandas as pd
import glob
import os
from pathlib import Path
import re
import pickle
import string
import numpy as np

In [5]:
from striprtf.striprtf import rtf_to_text

## Loading files into the df and extracting the names and year

In [6]:
# otwieranie wszystkich plikow w folderze i zapisanie ich do df

dir_path = "../data/sec_DEU/"

def get_docx_files(dir_path):
    docx_files = []
    
    for root, _, files in os.walk(dir_path):
        for f in files:
            if f.endswith('.rtf'):
                docx_files.append(os.path.join(root, f))
    
    return docx_files

docx_files = get_docx_files(dir_path)

data = []

# Iterate over all files in the directory
for filename in docx_files:
    if filename.endswith(".rtf"):
        with open(filename, 'rb') as file:
            rtf_content = file.read().decode('utf-8')  # Decode binary content to string
        # Parse the RTF content using pyth
            rtf_text = rtf_to_text(rtf_content)       
            data.append([filename[:-5],rtf_text])

# Create a pandas DataFrame from the extracted data
df = pd.DataFrame(data, columns=['f_name','raw_txt'])

In [7]:
len(df[df['raw_txt'].str.contains('Text of Article')])

36

In [13]:
df[df['f_name'].str.contains('SPIEGEL')]

,f_name,raw_txt
34,../data/sec_DEU/SPIEGEL ONLINE/Factiva-SPIEGEL...,\n\n\nBrandenburger Regierung startet freiwil...
35,../data/sec_DEU/SPIEGEL ONLINE/Factiva-SPIEGEL...,"\n\n\nTitel, Saporischschja, Impftote, Adichi..."
72,../data/sec_DEU/SPIEGEL/Factiva-SPIEGEL-01-202...,\n\n\nCoronakrise\n»Ich habe Angst vor den Me...


In [29]:
text=df.iloc[34]['raw_txt']
# Regular expression pattern for identifying the start of each article
pattern = r'\n\n\n.+?\n\n.+? \n\d+ Wörter\n\d{1,2} \w+ \d{4}\n.+\n.+\n\d+ \d+\n.+\n© \d+ .+? GmbH. All rights reserved. \n\n'

# Splitting the text using the pattern
articles = re.split(pattern, text)

In [32]:
df1=df[df['raw_txt'].str.contains('Text of Article')]

In [38]:
# Split the 'text' column by '***' and stack the resulting Series into a new DataFrame
split_df = df1['raw_txt'].str.split(r'\*\*\*', expand=True).stack().reset_index(level=1, drop=True).reset_index()

In [48]:
df2=split_df.dropna(subset=[0])

In [53]:
df2=df2[df2[0]!='']

In [54]:
df2

,index,0
1,49,\nDate of publication: 04/08/2021\nLink: https...
2,49,\nDate of publication: 04/08/2021\nLink: https...
3,49,\nDate of publication: 04/08/2021\nLink: https...
4,49,\nDate of publication: 07/08/2021\nLink: https...
5,49,\nDate of publication: 07/08/2021\nLink: https...
...,...,...
1115,84,\nDate of publication: 25/01/2021\nLink: https...
1116,84,\nDate of publication: 25/01/2021\nLink: https...
1117,84,\nDate of publication: 26/01/2021\nLink: https...
1118,84,\nDate of publication: 28/01/2021\nLink: https...


In [7]:
# df.to_csv('deu_sec_raw.csv')

In [34]:
# df['date']=df['f_name'].apply(lambda x: x.split('/')[5].split('_')[0])
# df['date']=df['date'].str[:7]
# df['year']=df['date'].str[:4]

In [60]:
# df['date']=np.where(df['date']=='2020-Is','2020-01',df['date'])
# df['date']=np.where(df['date']=='2021-Is','2021-01',df['date'])
# df['date']=np.where(df['date']=='2022-Is','2022-01',df['date'])

In [61]:
# df['date'] = df['date'].replace('0000-00-00', np.nan)
# df['date'] = pd.to_datetime(df['date'], format='%Y-%m')

In [62]:
# df['year'] = pd.to_datetime(df['year'], format='%Y', errors='coerce')

In [63]:
# zmiana kolejności, aby źródło było po roku
# df = df.iloc[:, [2,3,0,1]].copy()

In [8]:
# df.info()

In [56]:
# Function to extract text after "Text of Article:"
def extract_article_text(text):
    article_marker = "Text of Article:"
    article_index = text.find(article_marker)
    if article_index != -1:
        return text[article_index + len(article_marker):].strip()
    else:
        return text

# Apply the function to create the new column
df2['txt_1'] = df2[0].apply(extract_article_text)

/Users/michalpalinski/anaconda3/envs/myPython/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [57]:
df2

,index,0,txt_1
1,49,\nDate of publication: 04/08/2021\nLink: https...,Eine Impfpflicht muss man nicht gutheißen. Abe...
2,49,\nDate of publication: 04/08/2021\nLink: https...,Iranischer Präsident ins Amt eingeführt / USA ...
3,49,\nDate of publication: 04/08/2021\nLink: https...,Der ehemalige Präsident Barack Obama wird 60 J...
4,49,\nDate of publication: 07/08/2021\nLink: https...,Nirgends in Deutschland gibt es weniger Christ...
5,49,\nDate of publication: 07/08/2021\nLink: https...,Indonesisches Präsent für Badminton-Gold\nDie ...
...,...,...,...
1115,84,\nDate of publication: 25/01/2021\nLink: https...,Afrika erwartet von Biden mehr Respekt und Int...
1116,84,\nDate of publication: 25/01/2021\nLink: https...,Evangelische Theologen plädieren für einen pro...
1117,84,\nDate of publication: 26/01/2021\nLink: https...,Angesichts des tausendfachen Sterbens seien di...
1118,84,\nDate of publication: 28/01/2021\nLink: https...,Der Libanon braucht sein Nationalmuseum mehr d...


## text cleaning

In [58]:
def clean(text):
    text = re.sub("http\S+|www\S+|fb.com\S+|programm.ard.de\S+", '', text)
    text = ''.join([char for char in text if char not in '–!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~“”·‘'])
    text = re.sub('\n',' ',text)
    text = re.sub('‹|›|„','',text)
    text = re.sub(r'\b\d+\b\s*$', '', text)
    text = re.sub(r'\b\w*\d+\w*\b', '', text)
    text = re.sub(r'\b\d+\b', '', text)
    text = re.sub(r'\s{2,}', ' ', text)
    text = re.sub(r'•\t', '', text)
    return text.strip()

In [59]:
df2['cleaned_txt'] = df2['txt_1'].apply(clean)

/Users/michalpalinski/anaconda3/envs/myPython/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [60]:
df2['len_text'] = df2['txt_1'].apply(lambda x: len(x.split()))

/Users/michalpalinski/anaconda3/envs/myPython/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [61]:
# sprawdzam jak długi jest tekst i czy czegoś nie ucięło, wyrywkowo
df2['len_text'].describe()

count    1084.000000
mean      972.971402
std       699.643425
min         0.000000
25%       548.500000
50%       816.500000
75%      1224.000000
max      6819.000000
Name: len_text, dtype: float64

In [62]:
min_row = df.loc[df['len_text'].idxmin()]

# Print the row
print(min_row)

f_name                  ../data/sec_DEU/SZ/Factiva-SZ-22-2023091
raw_txt         \n\n\nFürstenfeldbruck\nTermine\n\n442 Wörter...
txt_1           \n\n\nFürstenfeldbruck\nTermine\n\n442 Wörter...
cleaned_txt    Fürstenfeldbruck Termine Wörter Februar Süddeu...
len_text                                                    6028
Name: 14, dtype: object


In [69]:
df2=df2[df2['len_text']>300]

In [70]:
df2.to_csv('../data/clean/sec_deu_clean.csv')